# Simulations using NFBC1966 results

Using chromosome 21 of the NFBC (Stampeed) data ($n=5340$ samples and $p = 5540$ SNPs), let us simulate:
+ Non-independent samples
+ Polygenic traits where every SNP contributes to the variance of the phenotypes, but only $k$ SNPs contribute to the mean of the phenotype.

## Simulation model

For $r$ traits, our simulation model is:

$$\mathbf{Y}_{r \times n} \sim \text{MatrixNormal}(\mathbf{B}_{r \times p}\mathbf{X}_{p \times n}, \ \ \Sigma_{r \times r} , \ \ \sigma_g^2\Phi_{n \times n} + \sigma_e^2 \mathbf{I}_{n \times n})$$

+ $\mathbf{X}_{p \times n}$ contains *all* predictors (genetic + non-genetic)
+ $\mathbf{B}_{r \times p}$ contains (true) regression coefficients. $k$ entries are non-zero
+ $\Phi$ is the GRM estimated from genotypes
+ $\Sigma_{r \times r} = \mathbf{A}^t\mathbf{A}$ is the row (trait) covariance matrix where $A_{ij} \sim N(0, 1)$
+ $\sigma_g^2\Phi_{n \times n} + \sigma_e^2 \mathbf{I}_{n \times n}$ is the column (sample) covariance matrix
+ $\sigma_g^2 = 0.6$ and $\sigma_e^2 = 0.4$ (thus heritability is 60%)
+ $k = \{10, 20, 50, 100\}$ (number of causal SNPs)
+ Pleiotripic overlap = 20% (e.g. if $k=10$, then 2 causal SNPs are shared in each trait, thus only 8 unique causal SNPs)
+ $\beta_i \sim N(0, 0.1)$
+ $r = \{2, 5\}$  (number of traits)
+ $n=5340$ samples
+ $p = 5540$ SNPs

In [1]:
# using Revise
# using MendelIHT
# using SnpArrays
# using Random
# using GLM
# using DelimitedFiles
# using Distributions
# using LinearAlgebra
# using CSV
# using DataFrames
# using StatsBase
# using Statistics
# using TraitSimulation

using Distributed
addprocs(8)
@everywhere begin
    using Revise
    using MendelIHT
    using SnpArrays
    using Random
    using GLM
    using DelimitedFiles
    using Distributions
    using LinearAlgebra
    using CSV
    using DataFrames
    using StatsBase
    using Statistics
    using TraitSimulation
end

┌ Info: Precompiling MendelIHT [921c7187-1484-5754-b919-5d3ed9ac03c4]
└ @ Base loading.jl:1317


### Helper functions for parsing results

In [2]:
"""
Find the `nsnps` most significant SNPs for mvPLINK in simulation `sim`.
"""
function get_top_mvPLINK_SNPs(sim::Int, nsnps::Int)
    n, p = 5402, 26906
    dir = "NFBCsim/sim$sim/"
    
    # read mvPLINK result
    mvplink_df = CSV.read(dir * "plink.mqfam.total", DataFrame, delim=' ', ignorerepeated=true)

    # get pvalues, possibly accounting for "NA"s
    if eltype(mvplink_df[!, :P]) == Float64
        pval = mvplink_df[!, :P]
    else
        mvplink_df[findall(x -> x == "NA", mvplink_df[!, :P]), :P] .= "1.0"
        pval = parse.(Float64, mvplink_df[!, :P])
    end
    perm = sortperm(pval)
    return perm[1:nsnps]
end

"""
Find the position of the `nsnps` most significant SNPs for GEMMA in simulation `sim`.
"""
function get_top_GEMMA_SNP_ids(sim::Int, nsnps::Int)
    dir = "NFBCsim/sim$sim/"
    gemma_df = CSV.read(dir * "gemma.sim$sim.assoc.txt", DataFrame)
    pval_wald = gemma_df[!, :p_wald]
    perm = sortperm(pval_wald)
    return perm[1:nsnps]
end

"""
Find significant SNPs return by IHT in simulation `sim`.
"""
function get_IHT_SNPs(sim::Int)
    dir = "NFBCsim/sim$sim/"
    iht_β1 = vec(readdlm(dir * "iht_beta1.txt"))
    iht_β2 = vec(readdlm(dir * "iht_beta2.txt"))
    detected_snps = findall(!iszero, iht_β1) ∪ findall(!iszero, iht_β2)
    return unique(detected_snps)
end

"""
Get positions for the truly causal SNPs in simulation `sim`. 
"""
function get_true_SNPs(sim::Int)
    model = "NFBCsim"
    dir = "NFBCsim/sim$sim/"
    trueB = readdlm(dir * "trueb.txt")
    causal_snps = unique([x[1] for x in findall(!iszero, trueB)])
    return causal_snps
end

"""
Get causal SNPs' position in GEMMA's result for simulation `sim`. Note gemma have snp filtering.
"""
function get_gemma_causal_snp_pos(sim::Int)
    model = "NFBCsim"
    nfbc = SnpData("/Users/biona001/Benjamin_Folder/UCLA/research/stampeed/imputed_with_0/NFBC_imputed_with_0")
    dir = "NFBCsim/sim$sim/"
    trueB = readdlm(dir * "trueb.txt")
    causal_snps = unique([x[1] for x in findall(!iszero, trueB)])
    causal_snp_rsID = nfbc.snp_info.snpid[causal_snps]
    gemma_df = CSV.read("NFBCsim/sim$sim/gemma.sim$sim.assoc.txt", DataFrame)
    gemma_snps = gemma_df[!, :rs]
    causal_snp_idx = convert(Vector{Int}, indexin(causal_snp_rsID, gemma_snps))
    
    # also need IHT's selected SNPs
    iht_snps_rsID = nfbc.snp_info.snpid[get_IHT_SNPs(sim)]
    iht_snps_idx = convert(Vector{Int}, indexin(iht_snps_rsID, gemma_snps))
    
    # also need SNP positions in GEMMA dataframe
    gemma2nfbc_idx = convert(Vector{Int}, indexin(gemma_snps, nfbc.snp_info.snpid))
    gemma_snp_pos = Vector{Int}(undef, size(gemma_df, 1))
    for i in 1:size(gemma_df, 1)
        gemma_snp_pos[i] = nfbc.snp_info.position[gemma2nfbc_idx[i]]
    end
    insertcols!(gemma_df, size(gemma_df, 2) + 1, :pos => gemma_snp_pos)
    
    return gemma_df, causal_snp_idx, iht_snps_idx
end

"""
- filename: gemma's output file name
- correct_snps: indices for real causal SNPs

returns: power, number of false positives, and false positive rate
"""
function process_gemma_result(filename, correct_snps)
    # read GEMMA result
    gemma_df = CSV.read(filename, DataFrame)
    snps = size(gemma_df, 1)

    # pvalues
    pval_wald = gemma_df[!, :p_wald]
#    pval_lrt = gemma_df[!, :p_lrt]
#     pval_score = gemma_df[!, :p_score]

    # estimated beta
    estim_β1 = gemma_df[!, :beta_1]
    estim_β2 = gemma_df[!, :beta_2]

    # estimated covariance matrix
    estim_σ11 = gemma_df[!, :Vbeta_1_1]
    estim_σ12 = gemma_df[!, :Vbeta_1_2]
    estim_σ22 = gemma_df[!, :Vbeta_2_2];

    # check how many real SNPs were recovered
    signif_snps = findall(x -> x ≤ 0.05 / snps, pval_wald) # gemma's selected snps
    signif_snps_rsID = gemma_df[signif_snps, :rs]
    
    # compute power, false positives, and false positive rate
    power_and_fpr(snps, correct_snps, signif_snps_rsID)
end

"""
- filename: mvPLINK's output file name
- correct_snps: indices for real causal SNPs

returns: power, number of false positives, and false positive rate
"""
function process_mvPLINK(filename, correct_snps)
    # read mvPLINK result
    mvplink_df = CSV.read(filename, DataFrame, delim=' ', ignorerepeated=true)
    snps = size(mvplink_df, 1)

    # get pvalues, possibly accounting for "NA"s
    if eltype(mvplink_df[!, :P]) == Float64
        pval = mvplink_df[!, :P]
    else
        mvplink_df[findall(x -> x == "NA", mvplink_df[!, :P]), :P] .= "1.0"
        pval = parse.(Float64, mvplink_df[!, :P])
    end

    # SNPs passing threshold
    signif_snps = findall(x -> x ≤ 0.05 / snps, pval)

    # compute power, false positives, and false positive rate
    power_and_fpr(snps, correct_snps, signif_snps)
end

"""
Computes power and false positive rates
- p: total number of SNPs
- correct_snps: Indices (or rsID) of the true causal SNPs
- detected_snps: Indices (or rsID) of SNPs that are significant after testing
"""
function power_and_fpr(p::Int, correct_snps::Vector, detected_snps::Vector)
    power = length(detected_snps ∩ correct_snps) / length(correct_snps)
    FP = length(detected_snps) - length(detected_snps ∩ correct_snps) # number of false positives
    TN = p - length(detected_snps) # number of true negatives
    FPR = FP / (FP + TN)
    return power, FP, FPR
end

"""
Summarize all simulations for IHT, mvPLINK, GEMMA in computation time, true positives,
false positives, and false positive rates. 
"""
function summarize_repeats()
    model = "NFBCsim"
    n, p = 5402, 26906
    sims = 1:50 # k = 10, r = 2, βoverlap=2, polygenic model
    nfbc = SnpData("/Users/biona001/Benjamin_Folder/UCLA/research/stampeed/imputed_with_0/NFBC_imputed_with_0")
    snp_rsID = nfbc.snp_info.snpid

    iht_time, iht_power, iht_FP, iht_FPR = Float64[], Float64[], Float64[], Float64[]
    mvPLINK_time, mvPLINK_power, mvPLINK_FP, mvPLINK_FPR = Float64[], Float64[], Float64[], Float64[]
    gemma_time, gemma_power, gemma_FP, gemma_FPR = Float64[], Float64[], Float64[], Float64[]

    for sim in sims
        dir = "NFBCsim/sim$sim/"
        try
            # correct SNPs
            trueB = readdlm(dir * "trueb.txt")
            causal_snps = unique([x[1] for x in findall(!iszero, trueB)])
            causal_snps_rsID = snp_rsID[causal_snps]

            # IHT
            iht_β1 = vec(readdlm(dir * "iht_beta1.txt"))
            iht_β2 = vec(readdlm(dir * "iht_beta2.txt"))
            detected_snps = findall(!iszero, iht_β1) ∪ findall(!iszero, iht_β2)
            ihtpower, ihtFP, ihtFPR = power_and_fpr(p, causal_snps, detected_snps)

            # MVPLINK
            plinkpower, plinkFP, plinkFPR = process_mvPLINK(dir * "plink.mqfam.total", causal_snps)

            # GEMMA 
            gemmapower, gemmaFP, gemmaFPR = process_gemma_result(dir * "gemma.sim$sim.assoc.txt", causal_snps_rsID)
            
            push!(iht_power, ihtpower); push!(iht_FP, ihtFP); push!(iht_FPR, ihtFPR); 
            push!(mvPLINK_power, plinkpower); push!(mvPLINK_FP, plinkFP); push!(mvPLINK_FPR, plinkFPR); 
            push!(gemma_power, gemmapower); push!(gemma_FP, gemmaFP); push!(gemma_FPR, gemmaFPR);
        catch
            println("simulation $sim failed!")
        end
    end

    return iht_time, iht_power, iht_FP, iht_FPR,
        mvPLINK_time, mvPLINK_power, mvPLINK_FP, mvPLINK_FPR,
        gemma_time, gemma_power, gemma_FP, gemma_FPR
end

"""
Imports gemma p-values, causal SNPs, and IHT selected SNP, and plot manhattan plot using MendelPlots.jl
"""
function plot_gemma_manhattan(sim::Int)
    # GEMMA causal SNPs
    gemma_df, causal_snps, iht_snps = get_gemma_causal_snp_pos(sim)
    rename!(gemma_df, [:p_wald => :pval, :rs => :snpid])
    gemma_df[findall(x -> x < 1e-50, gemma_df[!, :pval]), :pval] .= 1e-50
    empty_col = ["" for i in 1:size(gemma_df, 1)]
    insertcols!(gemma_df, size(gemma_df, 2) + 1, :empty_col => empty_col)

    manhattan(gemma_df, outfile = "NFBCsim/manhattan_gemma_sim$sim.png",
        annotateinds = causal_snps, annotateinds2 = iht_snps,
        annotatevar=:empty_col, titles="GEMMA simulation $sim")
    display("image/png", read("NFBCsim/manhattan_gemma_sim$sim.png"))
end

"""
Imports mvPLINK p-values, causal SNPs, and IHT selected SNP, and plot manhattan plot using MendelPlots.jl
"""
function plot_mvPLINK_manhattan(sim::Int)
    # mvPLINK
    filename = "NFBCsim/sim$sim/plink.mqfam.total"
    mvplink_df = CSV.read(filename, DataFrame, delim=' ', ignorerepeated=true)
    if eltype(mvplink_df[!, :P]) == Float64
        pval = mvplink_df[!, :P]
    else
        mvplink_df[findall(x -> x == "NA", mvplink_df[!, :P]), :P] .= "1.0"
        pval = parse.(Float64, mvplink_df[!, :P])
    end
    pval[findall(x -> x < 1e-50, pval)] .= 1e-50 # limit smallest pvalues

    # causal SNPs
    causal_snps = get_true_SNPs(sim)
    
    # IHT SNPs
    iht_snps = get_IHT_SNPs(sim)

    # make dataframe to input into MendelPlots
    snpdata = SnpData("/Users/biona001/Benjamin_Folder/UCLA/research/stampeed/imputed_with_0/NFBC_imputed_with_0")
    rename!(snpdata.snp_info, [:chr, :snpid, :genetic_distance, :pos, :allele1, :allele2])
    insertcols!(snpdata.snp_info, size(snpdata.snp_info, 2) + 1, :pval => pval)
    empty_col = ["" for i in 1:size(snpdata.snp_info, 1)]
    insertcols!(snpdata.snp_info, size(snpdata.snp_info, 2) + 1, :empty_col => empty_col)

    # plot
    manhattan(snpdata.snp_info, outfile = "NFBCsim/manhattan_mvPLINK_sim$sim.png",
        annotateinds = causal_snps, annotateinds2 = iht_snps, 
        annotatevar=:empty_col, titles="mvPLINK simulation $sim")
    display("image/png", read("NFBCsim/manhattan_mvPLINK_sim$sim.png"))
end

"""
Imports mvPLINK p-values and plot QQ plot using MendelPlots.jl
"""
function plot_mvPLINK_QQ(sim::Int)
    filename = "NFBCsim/sim$sim/plink.mqfam.total"
    mvplink_df = CSV.read(filename, DataFrame, delim=' ', ignorerepeated=true)
    if eltype(mvplink_df[!, :P]) == Float64
        pval = mvplink_df[!, :P]
    else
        mvplink_df[findall(x -> x == "NA", mvplink_df[!, :P]), :P] .= "1.0"
        pval = parse.(Float64, mvplink_df[!, :P])
    end
    pval[findall(x -> x < 1e-50, pval)] .= 1e-50 # limit smallest pvalues
    qq(pval, outfile = "NFBCsim/QQ_mvPLINK_sim$sim.png",
        ylabel="mvPLINK observed -log10(p)", titles="mvPLINK simulation $sim")
    display("image/png", read("NFBCsim/QQ_mvPLINK_sim$sim.png"))
end

"""
Imports gemma p-values and plot QQ plot using MendelPlots.jl
"""
function plot_gemma_QQ(sim::Int)
    filename = "NFBCsim/sim$sim/gemma.sim$sim.assoc.txt"
    gemma_df = CSV.read(filename, DataFrame)
    pval_wald = gemma_df[!, :p_wald]
    pval_wald[findall(x -> x < 1e-50, pval_wald)] .= 1e-50 # limit smallest pvalues
    qq(pval_wald, outfile = "NFBCsim/QQ_gemma_sim$sim.png",
        ylabel="GEMMA observed -log10(p)", titles="GEMMA simulation $sim")
    display("image/png", read("NFBCsim/QQ_gemma_sim$sim.png"))
end

"""
After performing n simulations using `run_repeats`, this function reads the summary files
for each simulation and summarizes the result. 
"""
function read_summary(simulations::UnitRange)
    model = "NFBCsim"
    
    iht_time, iht_power, iht_FP, iht_FPT = Float64[], Float64[], Float64[], Float64[]
    mvPLINK_time, mvPLINK_power, mvPLINK_FP, mvPLINK_FPT = Float64[], Float64[], Float64[], Float64[]
    gemma_time, gemma_power, gemma_FP, gemma_FPT = Float64[], Float64[], Float64[], Float64[]

    regex = r"= (\d+\.\d+) seconds, power = (\d+\.\d+), FP = (\d+), FPR = (\d\.\d+e?-?\d*)"
    for sim in simulations
        if !isdir("$model/sim$sim") || !isfile("$model/sim$(sim)/summary.txt")
            println("Simulation $sim failed!")
            continue
        end
        try
            open("$model/sim$(sim)/summary.txt", "r") do io
                readline(io); readline(io); readline(io)

                # parse IHT result
                iht = match(regex, readline(io))
                push!(iht_time, parse(Float64, iht[1]))
                push!(iht_power, parse(Float64, iht[2]))
                push!(iht_FP, parse(Float64, iht[3]))
                push!(iht_FPT, parse(Float64, iht[4]))

                # parse mvPLINK result
                mvPLINK = match(regex, readline(io))
                push!(mvPLINK_time, parse(Float64, mvPLINK[1]))
                push!(mvPLINK_power, parse(Float64, mvPLINK[2]))
                push!(mvPLINK_FP, parse(Float64, mvPLINK[3]))
                push!(mvPLINK_FPT, parse(Float64, mvPLINK[4]))

                # parse mvPLINK result
                gemma = match(regex, readline(io))
                push!(gemma_time, parse(Float64, gemma[1]))
                push!(gemma_power, parse(Float64, gemma[2]))
                push!(gemma_FP, parse(Float64, gemma[3]))
                push!(gemma_FPT, parse(Float64, gemma[4]))
            end
        catch
            println("Simulation $sim failed!")
            continue
        end
    end

    # save summary statistics
    open("$(model)_summary.txt", "w") do io
        println(io, "iht_time,iht_power,iht_FP,iht_FPT,mvPLINK_time,mvPLINK_power," * 
            "mvPLINK_FP,mvPLINK_FPT,gemma_time,gemma_power,gemma_FP,gemma_FPT")
        for i in eachindex(iht_time)
            print(io, iht_time[i], ',', iht_power[i], ',', iht_FP[i], ',', iht_FPT[i], ',')
            print(io, mvPLINK_time[i], ',', mvPLINK_power[i], ',', mvPLINK_FP[i], ',', mvPLINK_FPT[i], ',')
            print(io, gemma_time[i], ',', gemma_power[i], ',', gemma_FP[i], ',', gemma_FPT[i], "\n")
        end
    end
    
    return iht_time, iht_power, iht_FP, iht_FPT, 
        mvPLINK_time, mvPLINK_power, mvPLINK_FP, mvPLINK_FPT,
        gemma_time, gemma_power, gemma_FP, gemma_FPT
end

"""
k = Number of causal SNPs
p = Total number of SNPs
traits = Number of traits (phenotypes)
overlap = number of causal SNPs shared in each trait
"""
function simulate_random_beta(k::Int, p::Int, traits::Int; overlap::Int=0, βσ=1.0)
    d = Normal(0, βσ)
    true_b = zeros(p, traits)
    if overlap == 0
        causal_snps = sample(1:(traits * p), k, replace=false)
        true_b[causal_snps] = rand(d, k)
    else
        shared_snps = sample(1:p, overlap, replace=false)
        weight_vector = aweights(1 / (traits * (p - overlap)) * ones(traits * p))
        for i in 1:traits
            weight_vector[i*shared_snps] .= 0.0 # avoid sampling from shared snps
        end
        @assert sum(weight_vector) ≈ 1.0
        # simulate β for shared predictors
        for i in 1:traits
            true_b[shared_snps, i] = rand(d, overlap)
        end
        # simulate β for none shared predictors
        nonshared_snps = sample(1:(traits * p), weight_vector, k - (traits * overlap), replace=false)
        true_b[nonshared_snps] = rand(d, k - (traits * overlap))
    end

    return true_b
end

"""
ad-hoc matrix-matrix multiplication (out = st * v) using SnpLinAlg
"""
function adhoc_mul!(
    out::AbstractMatrix{T}, 
    st::SnpLinAlg{T},
    v::AbstractMatrix{T}) where T <: AbstractFloat
    @assert size(out, 1) == size(st, 1) && size(v, 2) == size(v, 2) && size(st, 2) == size(v, 1)
    for i in 1:size(v, 2)
        outi = @view(out[:, i])
        vi = @view(v[:, i])
        SnpArrays.mul!(outi, st, vi)
    end
end

function simulate_NFBC1966_polygenic(
    plinkname::String, k::Int, r::Int;
    seed::Int=2021, σg=0.6, σe=0.4, βoverlap=2, βσ=1.0,
    )
    # set seed
    Random.seed!(seed)
    
    # simulate `.bed` file with no missing data
    x = SnpArray(plinkname * ".bed")
#     xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, impute=true, center=true, scale=true)
    xla = convert(Matrix{Float64}, x, impute=true, center=true, scale=true)
    n, p = size(x)
    
    # intercept is the only nongenetic covariate
    Z = ones(n, 1)
    intercepts = randn(r)' # each trait have different intercept

    # simulate β
    B = simulate_random_beta(k, p, r, overlap=βoverlap, βσ=βσ)

    # between trait covariance matrix
    Σ = random_covariance_matrix(r)

    # between sample covariance is identity + GRM
    Φ = grm(x, method=:Robust)
    V = σg * Φ + σe * I

    # simulate y using TraitSimulations.jl (https://github.com/OpenMendel/TraitSimulation.jl/blob/master/src/modelframework.jl#L137)
    vc = @vc Σ ⊗ V
    μ = zeros(n, r)
    μ_null = zeros(n, r)
    LinearAlgebra.mul!(μ_null, Z, intercepts)
#     adhoc_mul!(μ, xla, B)
    mul!(μ, xla, B)
    BLAS.axpby!(1.0, μ_null, 1.0, μ)
    VCM_model = VCMTrait(Z, intercepts, xla, B, vc, μ)
    Y = Matrix(Transpose(simulate(VCM_model)))
    
    return xla, Matrix(Z'), B, Σ, Y
end

function simulate_NFBC1966_sparse(
    plinkname::String, k::Int, r::Int;
    seed::Int=2021, βoverlap=2, βσ = 1.0
    )
    # set seed
    Random.seed!(seed)
    
    # simulate `.bed` file with no missing data
    x = SnpArray(plinkname * ".bed")
#     xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true)
    xla = convert(Matrix{Float64}, x, impute=true, center=true, scale=true)
    n, p = size(xla)
    
    # intercept is the only nongenetic covariate
    Z = ones(n, 1)
    intercepts = zeros(r)' # each trait have different intercept

    # simulate β
    B = simulate_random_beta(k, p, r, overlap=βoverlap, βσ=βσ)

    # between trait covariance matrix
    Σ = random_covariance_matrix(r)
    
    # simulate multivariate normal phenotype for each sample
    μ = xla * B + Z*intercepts

    # simulate response
    Y = zeros(n, r)
    for i in 1:n
        μi = @view(μ[i, :])
        Y[i, :] = rand(MvNormal(μi, Σ))
    end
    
    return xla, Matrix(Z'), B, Σ, Matrix(Y')
end

function simulate_multivariate_sparse(
    n::Int, p::Int, k::Int, r::Int;
    seed::Int=2021, βoverlap=2, 
    )
    # set seed
    Random.seed!(seed)
    
    # simulate `.bed` file with no missing data
    x = simulate_random_snparray(undef, n, p)
    xla = convert(Matrix{Float64}, x, impute=true, center=true, scale=true)
#     xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true)
    n, p = size(x)

    # intercept is the only nongenetic covariate
    z = ones(n, 1)
    intercepts = zeros(r)' # each trait have different intercept

    # simulate response y, true model b, and the correct non-0 positions of b
    Y, true_Σ, true_b, correct_position = simulate_random_response(xla, k, r, Zu=z*intercepts, overlap=βoverlap);
    
    return xla, Matrix(z'), true_b, true_Σ, Matrix(Y')
end

simulate_multivariate_sparse (generic function with 1 method)

# Try running IHT

In [3]:
# simulation parameters
k = 10
r = 2
βσ = 0.1
seed = 1111
βoverlap = 2

# use chr21 of NFBC
chr21 = "/Users/biona001/Benjamin_Folder/UCLA/research/stampeed/qc/NFBC.qc.chr.21"
xla, Z, true_b, Σ, Y = simulate_NFBC1966_sparse(chr21,k,r,seed=seed,βoverlap=βoverlap,βσ=βσ)
causal_snps = unique([x[1] for x in findall(!iszero, true_b)]);

# simulate random data
# n = 1000
# p = 10000
# xla, Z, true_b, Σ, Y = simulate_multivariate_sparse(n, p, k,r,seed=seed,βoverlap=βoverlap)
# causal_snps = unique([x[1] for x in findall(!iszero, true_b)]);

In [4]:
println(Σ)
println(causal_snps)
true_b[causal_snps, :]

[0.056419962055242315 0.1802121298258207; 0.1802121298258207 2.723094788677102]
[2349, 3073, 3075, 4653, 5091, 5344, 5485, 2651]


8×2 Matrix{Float64}:
 -0.0523363   0.146389
 -0.105674    0.0
 -0.03193     0.0
  0.0654857   0.0
  0.0360225   0.0
 -0.0429316   0.0
  0.108861    0.0877691
  0.0        -0.141978

In [5]:
# no intercept term
function loglikelihood(X, Y, B, Γ)
    n = size(Y, 2)
    resid = Y - B*X
    return n/2 * logdet(Γ) - 0.5 * tr(Γ*resid*resid')
end

# this should be the best loglikehood attainable 
@show loglikelihood(Transpose(xla), Y, Transpose(true_b), inv(Σ))

# this should be loglikehood at iter 1 
@show loglikelihood(Transpose(xla), Y, zeros(size(Transpose(true_b))), [1.0 0.0; 0.0 1.0])

loglikelihood(Transpose(xla), Y, Transpose(true_b), inv(Σ)) = 373.8599630859753
loglikelihood(Transpose(xla), Y, zeros(size(Transpose(true_b))), [1.0 0.0; 0.0 1.0]) = -7480.311995286563


-7480.311995286563

In [6]:
result = fit_iht(Y, Transpose(xla), k=12, init_beta=true, verbose=true, debias=true)

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter (k) = 12
Prior weight scaling = off
Doubly sparse projection = off
Debias = on
Max IHT iterations = 200
Converging when tol < 0.0001:

Iteration 1: loglikelihood = -348.5823880579728, backtracks = 0, tol = 0.0
Iteration 2: loglikelihood = -332.87462055781543, backtracks = 0, tol = 0.003647462926161623
Iteration 3: loglikelihood = -319.6504770864258, backtracks = 0, tol = 0.0033028925479674266
Iteration 4: loglikelihood = -308.2404972425411, backtracks = 0, tol = 0.003008504931047995



Compute time (sec):     0.5610418319702148
Final loglikelihood:    -80.25464250882578
Iterations:             7
Trait 1's SNP PVE:      0.25675536913268326
Trait 2's SNP PVE:      0.01544428569602941

Estimated trait covariance:
2×2 DataFrame
 Row │ trait1    trait2   
     │ Float64   Float64  
─────┼────────────────────
   1 │ 0.064432  0.180975
   2 │ 0.180975  2.67285

Trait 1: IHT estimated 3 nonzero SNP predictors
3×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │     2349   -0.0501577
   2 │     3073   -0.0953751
   3 │     5485    0.106728

Trait 1: IHT estimated 0 non-genetic predictors
0×2 DataFrame

Trait 2: IHT estimated 9 nonzero SNP predictors
9×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │     2345   -0.034452
   2 │     2346    0.171455
   3 │     2348   -0.191877
   4 │     2349    0.104694
   5 │     2353    0.060107
   6 │     2651   -0.0795871


In [7]:
result = fit_iht(Y, Transpose(xla), k=12, verbose=true, debias=true)

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter (k) = 12
Prior weight scaling = off
Doubly sparse projection = off
Debias = on
Max IHT iterations = 200
Converging when tol < 0.0001:

Iteration 1: loglikelihood = -1086.7080446381206, backtracks = 0, tol = 0.0
Iteration 2: loglikelihood = -1083.8182233107045, backtracks = 0, tol = 7.068275112573435e-5
Iteration 3: loglikelihood = -176.068152614067, backtracks = 0, tol = 0.035389719786740916
Iteration 4: loglikelihood = -47.66422231391789, backtracks = 0, tol = 0.025845269446551987
I


Compute time (sec):     0.7363009452819824
Final loglikelihood:    355.59977418903236
Iterations:             9
Trait 1's SNP PVE:      0.39179817428213126
Trait 2's SNP PVE:      0.008613478375068375

Estimated trait covariance:
2×2 DataFrame
 Row │ trait1     trait2   
     │ Float64    Float64  
─────┼─────────────────────
   1 │ 0.0562832  0.179476
   2 │ 0.179476   2.69174

Trait 1: IHT estimated 7 nonzero SNP predictors
7×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │     2349   -0.0421664
   2 │     3073   -0.103315
   3 │     3077    0.0335914
   4 │     4653    0.0661619
   5 │     5091    0.04317
   6 │     5344   -0.0359141
   7 │     5485    0.105314

Trait 1: IHT estimated 0 non-genetic predictors
0×2 DataFrame

Trait 2: IHT estimated 5 nonzero SNP predictors
5×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │     2349    0.0898681
   2 │     2350    0.

In [8]:
# beta 1
[result.beta[1, causal_snps] true_b[causal_snps, 1]]

8×2 Matrix{Float64}:
 -0.0421664  -0.0523363
 -0.103315   -0.105674
  0.0        -0.03193
  0.0661619   0.0654857
  0.04317     0.0360225
 -0.0359141  -0.0429316
  0.105314    0.108861
  0.0         0.0

In [9]:
# beta 2
[result.beta[2, causal_snps] true_b[causal_snps, 2]]

8×2 Matrix{Float64}:
 0.0898681   0.146389
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0265674   0.0877691
 0.0        -0.141978

In [10]:
# number of true positives
count(!iszero, sum(result.beta[:, causal_snps], dims=1))

6

In [11]:
# covariance
[vec(result.Σ) vec(Σ)]

4×2 Matrix{Float64}:
 0.0562832  0.05642
 0.179476   0.180212
 0.179476   0.180212
 2.69174    2.72309

In [12]:
causal_snps

8-element Vector{Int64}:
 2349
 3073
 3075
 4653
 5091
 5344
 5485
 2651

## Try 20 simulations (NFBC data, no debias)

We will compare

+ Number of true positives
+ Number of iteration until convergence
+ Best cross validated k

in scenario where we initialize beta values, versus not initializing. 

In [13]:
function bench(sim::Int)
    # simulation parameters
    k = 10
    r = 2
    βσ = 0.1
    βoverlap = 2
    chr21 = "/Users/biona001/Benjamin_Folder/UCLA/research/stampeed/qc/NFBC.qc.chr.21"

    no_init_TP = zeros(sim)
    no_init_iter = zeros(sim)
    no_init_best_k = zeros(sim)
    yes_init_TP = zeros(sim)
    yes_init_iter = zeros(sim)
    yes_init_best_k = zeros(sim)

    # use chr21 of NFBC
    for i in 1:sim
        xla, Z, true_b, Σ, Y = simulate_NFBC1966_sparse(chr21,k,r,seed=i,βoverlap=βoverlap,βσ=βσ)
        causal_snps = unique([x[1] for x in findall(!iszero, true_b)]);

        # not initializing beta
        mses = cv_iht(Y, Transpose(xla), path=1:20, verbose=true)
        result = fit_iht(Y, Transpose(xla), k=argmin(mses), verbose=true)
        no_init_TP[i] = count(!iszero, sum(result.beta[:, causal_snps], dims=1))
        no_init_iter[i] = result.iter
        no_init_best_k[i] = argmin(mses)

        # initializing beta
        mses = cv_iht(Y, Transpose(xla), path=1:20, init_beta=true)
        result = fit_iht(Y, Transpose(xla), k=argmin(mses), verbose=true, init_beta = true)
        yes_init_TP[i] = count(!iszero, sum(result.beta[:, causal_snps], dims=1))
        yes_init_iter[i] = result.iter
        yes_init_best_k[i] = argmin(mses)
    end

    return no_init_TP, no_init_iter, no_init_best_k, 
        yes_init_TP, yes_init_iter, yes_init_best_k
end
@time no_init_TP, no_init_iter, no_init_best_k, 
    yes_init_TP, yes_init_iter, yes_init_best_k = bench(20)

Cross validating...100%|████████████████████████████████| Time: 0:04:28




Crossvalidation Results:
	k	MSE
	1	7467.007578213517
	2	7467.016480222095
	3	7462.504268879455
	4	7462.414105189358
	5	7462.562881519958
	6	7462.729194037671
	7	7462.996119308439
	8	7463.0675206944925
	9	7463.05681877081
	10	7463.446247491652
	11	7463.47232982508
	12	7463.894828904651
	13	7463.66907552382
	14	7464.4044117643725
	15	7463.871210756556
	16	7464.021754154152
	17	7464.356385220899
	18	7465.448424125442
	19	7464.926936940883
	20	7465.868263113456

Best k = 4

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter (k) = 4
Pri

Cross validating...100%|████████████████████████████████| Time: 0:05:27




Crossvalidation Results:
	k	MSE
	1	7458.260827330637
	2	7463.298120100859
	3	7474.3830244782
	4	7484.012558113514
	5	7486.173844259777
	6	7505.856346671087
	7	7501.784218116077
	8	7503.162459082315
	9	7513.37364831712
	10	7520.165486197674
	11	7529.161697023144
	12	7535.050121376507
	13	7542.672857041491
	14	7556.781802997405
	15	7567.135434435953
	16	7576.9031707688455
	17	7576.311293515613
	18	7590.36611846913
	19	7594.052264475409
	20	7593.668400750834

Best k = 1

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter (k) = 1
Prior

Cross validating...100%|████████████████████████████████| Time: 0:05:24


...

Cross validating...100%|████████████████████████████████| Time: 0:01:54




Crossvalidation Results:
	k	MSE
	1	2686.6738771389537
	2	2687.204500771152
	3	2601.4461530447093
	4	2586.1952191880564
	5	2572.908756982094
	6	2558.1455456151716
	7	2537.2202006946604
	8	2543.343077831648
	9	2539.8595711934495
	10	2545.692966546877
	11	2539.7962982620543
	12	2543.6049785581486
	13	2542.8371883951163
	14	2544.320314405359
	15	2542.4478626452405
	16	2546.178667846294
	17	2546.363660841593
	18	2547.8129669128934
	19	2545.5213173203456
	20	2555.882300533487

Best k = 7

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity paramete

Cross validating...100%|████████████████████████████████| Time: 0:02:01




Crossvalidation Results:
	k	MSE
	1	2668.9841390603888
	2	2620.0537339033417
	3	2587.264582090639
	4	2578.456026836089
	5	2569.007540305888
	6	2559.160507263263
	7	2545.0488520940157
	8	2550.137120472825
	9	2547.4397347986114
	10	2544.9880040224252
	11	2545.0214508474023
	12	2549.0853567032086
	13	2550.754531299288
	14	2546.09954150562
	15	2546.7708643945925
	16	2549.871696812181
	17	2551.6090044367115
	18	2554.7844451517853
	19	2550.076662582116
	20	2554.3757441711987

Best k = 10

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter

([3.0, 6.0, 5.0, 3.0, 5.0, 4.0, 6.0, 5.0, 7.0, 1.0, 4.0, 4.0, 4.0, 6.0, 8.0, 5.0, 2.0, 4.0, 3.0, 5.0], [200.0, 200.0, 63.0, 9.0, 17.0, 200.0, 200.0, 44.0, 200.0, 142.0, 9.0, 12.0, 37.0, 25.0, 200.0, 18.0, 15.0, 14.0, 8.0, 35.0], [4.0, 13.0, 13.0, 3.0, 5.0, 8.0, 14.0, 6.0, 15.0, 5.0, 4.0, 4.0, 10.0, 8.0, 20.0, 5.0, 3.0, 4.0, 4.0, 7.0], [0.0, 6.0, 5.0, 3.0, 6.0, 4.0, 7.0, 7.0, 7.0, 1.0, 4.0, 4.0, 4.0, 4.0, 6.0, 4.0, 1.0, 4.0, 3.0, 6.0], [121.0, 158.0, 94.0, 12.0, 32.0, 200.0, 76.0, 127.0, 102.0, 67.0, 7.0, 10.0, 23.0, 16.0, 24.0, 16.0, 18.0, 17.0, 5.0, 41.0], [1.0, 14.0, 11.0, 3.0, 6.0, 7.0, 14.0, 15.0, 9.0, 2.0, 4.0, 4.0, 5.0, 5.0, 6.0, 5.0, 2.0, 4.0, 3.0, 10.0])

In [14]:
@show mean(no_init_TP)
@show mean(yes_init_TP)
[no_init_TP yes_init_TP]

mean(no_init_TP) = 4.5
mean(yes_init_TP) = 4.3


20×2 Matrix{Float64}:
 3.0  0.0
 6.0  6.0
 5.0  5.0
 3.0  3.0
 5.0  6.0
 4.0  4.0
 6.0  7.0
 5.0  7.0
 7.0  7.0
 1.0  1.0
 4.0  4.0
 4.0  4.0
 4.0  4.0
 6.0  4.0
 8.0  6.0
 5.0  4.0
 2.0  1.0
 4.0  4.0
 3.0  3.0
 5.0  6.0

In [15]:
@show mean(no_init_iter)
@show mean(yes_init_iter)
[no_init_iter yes_init_iter]

mean(no_init_iter) = 82.4
mean(yes_init_iter) = 58.3


20×2 Matrix{Float64}:
 200.0  121.0
 200.0  158.0
  63.0   94.0
   9.0   12.0
  17.0   32.0
 200.0  200.0
 200.0   76.0
  44.0  127.0
 200.0  102.0
 142.0   67.0
   9.0    7.0
  12.0   10.0
  37.0   23.0
  25.0   16.0
 200.0   24.0
  18.0   16.0
  15.0   18.0
  14.0   17.0
   8.0    5.0
  35.0   41.0

In [16]:
@show mean(no_init_best_k)
@show mean(yes_init_best_k)
[no_init_best_k yes_init_best_k]

mean(no_init_best_k) = 7.75
mean(yes_init_best_k) = 6.5


20×2 Matrix{Float64}:
  4.0   1.0
 13.0  14.0
 13.0  11.0
  3.0   3.0
  5.0   6.0
  8.0   7.0
 14.0  14.0
  6.0  15.0
 15.0   9.0
  5.0   2.0
  4.0   4.0
  4.0   4.0
 10.0   5.0
  8.0   5.0
 20.0   6.0
  5.0   5.0
  3.0   2.0
  4.0   4.0
  4.0   3.0
  7.0  10.0

## Try 20 simulations (NFBC data, debias)

We will compare

+ Number of true positives
+ Number of iteration until convergence
+ Best cross validated k

in scenario where we initialize beta values, versus not initializing. 

In [17]:
function bench(sim::Int)
    # simulation parameters
    k = 10
    r = 2
    βσ = 0.1
    βoverlap = 2
    chr21 = "/Users/biona001/Benjamin_Folder/UCLA/research/stampeed/qc/NFBC.qc.chr.21"

    no_init_TP = zeros(sim)
    no_init_iter = zeros(sim)
    no_init_best_k = zeros(sim)
    yes_init_TP = zeros(sim)
    yes_init_iter = zeros(sim)
    yes_init_best_k = zeros(sim)

    # use chr21 of NFBC
    for i in 1:sim
        xla, Z, true_b, Σ, Y = simulate_NFBC1966_sparse(chr21,k,r,seed=i,βoverlap=βoverlap,βσ=βσ)
        causal_snps = unique([x[1] for x in findall(!iszero, true_b)]);

        # not initializing beta
        mses = cv_iht(Y, Transpose(xla), path=1:20, verbose=true, debias=true)
        result = fit_iht(Y, Transpose(xla), k=argmin(mses), verbose=true, debias=true)
        no_init_TP[i] = count(!iszero, sum(result.beta[:, causal_snps], dims=1))
        no_init_iter[i] = result.iter
        no_init_best_k[i] = argmin(mses)

        # initializing beta
        mses = cv_iht(Y, Transpose(xla), path=1:20, init_beta=true, debias=true)
        result = fit_iht(Y, Transpose(xla), k=argmin(mses), verbose=true, init_beta = true, debias=true)
        yes_init_TP[i] = count(!iszero, sum(result.beta[:, causal_snps], dims=1))
        yes_init_iter[i] = result.iter
        yes_init_best_k[i] = argmin(mses)
    end

    return no_init_TP, no_init_iter, no_init_best_k, 
        yes_init_TP, yes_init_iter, yes_init_best_k
end
@time no_init_TP, no_init_iter, no_init_best_k, 
    yes_init_TP, yes_init_iter, yes_init_best_k = bench(20)

Cross validating...100%|████████████████████████████████| Time: 0:00:19




Crossvalidation Results:
	k	MSE
	1	7467.007578213517
	2	7467.016480222095
	3	7462.686520010227
	4	7459.731399434859
	5	7459.571145178593
	6	7457.612883138536
	7	7460.606294499556
	8	7456.681759241473
	9	7453.695689317448
	10	7453.483260738049
	11	7450.948727887585
	12	7449.373132917843
	13	7454.041410951184
	14	7453.081947522708
	15	7446.562437949809
	16	7448.961342829954
	17	7444.609016101175
	18	7439.239665478103
	19	7438.558642497654
	20	7446.166923049819

Best k = 19

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter (k) = 19


Cross validating...100%|████████████████████████████████| Time: 0:00:26




Crossvalidation Results:
	k	MSE
	1	7458.687808733434
	2	7461.704152142922
	3	7464.707553434446
	4	7476.675745851983
	5	7476.437905561855
	6	7483.87242299432
	7	7463.085155403782
	8	7465.324305579339
	9	7463.984127476909
	10	7470.118980054739
	11	7477.338987273931
	12	7482.055611463823
	13	7483.012940730736
	14	7485.348313617027
	15	7499.932601590906
	16	7507.263813269407
	17	7503.940842453529
	18	7502.954556004397
	19	7499.774021815997
	20	7494.422615125665

Best k = 1

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter (k) = 1
Pri

Cross validating...100%|████████████████████████████████| Time: 0:00:24


...

Cross validating...100%|████████████████████████████████| Time: 0:00:40




Crossvalidation Results:
	k	MSE
	1	2686.6738771389537
	2	2687.204500771152
	3	2599.7419730253623
	4	2585.257303093636
	5	2583.9553314944706
	6	2536.6187805458276
	7	2538.732292512429
	8	2532.2152682466485
	9	2533.2859811473627
	10	2529.441151030224
	11	2531.9270473915813
	12	2531.962556279564
	13	2528.3144870973556
	14	2532.5943010057294
	15	2531.2001200887753
	16	2534.2120445149667
	17	2532.7510173579412
	18	2529.9254760478634
	19	2527.3355893059525
	20	2530.643568366261

Best k = 19

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity param

Cross validating...100%|████████████████████████████████| Time: 0:00:38




Crossvalidation Results:
	k	MSE
	1	2640.015342861501
	2	2597.444581892266
	3	2567.799028107913
	4	2555.4190585670262
	5	2545.6170246686174
	6	2538.40334414925
	7	2533.4124775160617
	8	2533.1980627196676
	9	2532.6507057682197
	10	2529.6061423939414
	11	2532.0562754920093
	12	2533.011201771853
	13	2533.704580638236
	14	2531.407525970939
	15	2531.681180638196
	16	2530.0335173547546
	17	2532.6924857140066
	18	2532.282491110858
	19	2534.2084000976615
	20	2528.9678298650306

Best k = 20

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter

([3.0, 3.0, 6.0, 3.0, 5.0, 4.0, 5.0, 5.0, 5.0, 2.0, 5.0, 6.0, 3.0, 6.0, 8.0, 5.0, 3.0, 4.0, 3.0, 6.0], [13.0, 11.0, 8.0, 7.0, 10.0, 9.0, 13.0, 15.0, 11.0, 8.0, 12.0, 25.0, 12.0, 13.0, 10.0, 16.0, 9.0, 11.0, 8.0, 12.0], [19.0, 17.0, 20.0, 11.0, 19.0, 12.0, 18.0, 16.0, 15.0, 10.0, 20.0, 20.0, 18.0, 19.0, 20.0, 16.0, 6.0, 20.0, 8.0, 19.0], [0.0, 5.0, 5.0, 3.0, 5.0, 4.0, 7.0, 4.0, 4.0, 2.0, 4.0, 5.0, 4.0, 6.0, 7.0, 5.0, 1.0, 5.0, 3.0, 5.0], [6.0, 8.0, 9.0, 9.0, 7.0, 7.0, 11.0, 7.0, 9.0, 10.0, 200.0, 14.0, 16.0, 9.0, 11.0, 9.0, 7.0, 11.0, 6.0, 10.0], [1.0, 18.0, 19.0, 20.0, 14.0, 15.0, 15.0, 17.0, 9.0, 11.0, 17.0, 11.0, 10.0, 19.0, 20.0, 20.0, 2.0, 14.0, 3.0, 20.0])

In [19]:
@show mean(no_init_TP)
@show mean(yes_init_TP)
[no_init_TP yes_init_TP]

mean(no_init_TP) = 4.5
mean(yes_init_TP) = 4.2


20×2 Matrix{Float64}:
 3.0  0.0
 3.0  5.0
 6.0  5.0
 3.0  3.0
 5.0  5.0
 4.0  4.0
 5.0  7.0
 5.0  4.0
 5.0  4.0
 2.0  2.0
 5.0  4.0
 6.0  5.0
 3.0  4.0
 6.0  6.0
 8.0  7.0
 5.0  5.0
 3.0  1.0
 4.0  5.0
 3.0  3.0
 6.0  5.0

In [20]:
@show mean(no_init_iter)
@show mean(yes_init_iter)
[no_init_iter yes_init_iter]

mean(no_init_iter) = 11.65
mean(yes_init_iter) = 18.8


20×2 Matrix{Float64}:
 13.0    6.0
 11.0    8.0
  8.0    9.0
  7.0    9.0
 10.0    7.0
  9.0    7.0
 13.0   11.0
 15.0    7.0
 11.0    9.0
  8.0   10.0
 12.0  200.0
 25.0   14.0
 12.0   16.0
 13.0    9.0
 10.0   11.0
 16.0    9.0
  9.0    7.0
 11.0   11.0
  8.0    6.0
 12.0   10.0

In [18]:
@show mean(no_init_best_k)
@show mean(yes_init_best_k)
[no_init_best_k yes_init_best_k]

mean(no_init_best_k) = 16.15
mean(yes_init_best_k) = 13.75


20×2 Matrix{Float64}:
 19.0   1.0
 17.0  18.0
 20.0  19.0
 11.0  20.0
 19.0  14.0
 12.0  15.0
 18.0  15.0
 16.0  17.0
 15.0   9.0
 10.0  11.0
 20.0  17.0
 20.0  11.0
 18.0  10.0
 19.0  19.0
 20.0  20.0
 16.0  20.0
  6.0   2.0
 20.0  14.0
  8.0   3.0
 19.0  20.0

## Try 20 simulations (simulated genotypes, no debias)

We will compare

+ Number of true positives
+ Number of iteration until convergence
+ Best cross validated k

in scenario where we initialize beta values, versus not initializing. 

In [21]:
function bench(sim::Int)
    # simulation parameters
    n = 1000
    p = 10000
    k = 10
    r = 2
    βσ = 0.1
    βoverlap = 2

    no_init_TP = zeros(sim)
    no_init_iter = zeros(sim)
    no_init_best_k = zeros(sim)
    yes_init_TP = zeros(sim)
    yes_init_iter = zeros(sim)
    yes_init_best_k = zeros(sim)

    for i in 1:sim
        xla, Z, true_b, Σ, Y = simulate_multivariate_sparse(n, p, k,r,seed=i,βoverlap=βoverlap)
        causal_snps = unique([x[1] for x in findall(!iszero, true_b)])

        # not initializing beta
        mses = cv_iht(Y, Transpose(xla), path=1:20, verbose=true)
        result = fit_iht(Y, Transpose(xla), k=argmin(mses), verbose=true)
        no_init_TP[i] = count(!iszero, sum(result.beta[:, causal_snps], dims=1))
        no_init_iter[i] = result.iter
        no_init_best_k[i] = argmin(mses)

        # initializing beta
        mses = cv_iht(Y, Transpose(xla), path=1:20, init_beta=true)
        result = fit_iht(Y, Transpose(xla), k=argmin(mses), verbose=true, init_beta = true)
        yes_init_TP[i] = count(!iszero, sum(result.beta[:, causal_snps], dims=1))
        yes_init_iter[i] = result.iter
        yes_init_best_k[i] = argmin(mses)
    end

    return no_init_TP, no_init_iter, no_init_best_k, 
        yes_init_TP, yes_init_iter, yes_init_best_k
end
@time no_init_TP, no_init_iter, no_init_best_k, 
    yes_init_TP, yes_init_iter, yes_init_best_k = bench(20)

Cross validating...100%|████████████████████████████████| Time: 0:00:50




Crossvalidation Results:
	k	MSE
	1	3106.6316936694793
	2	3111.8676511084177
	3	2278.6561381609126
	4	2206.1907681338503
	5	1993.2244326438577
	6	1997.381409111596
	7	2014.1655251889138
	8	2016.840663407312
	9	2021.3604206629525
	10	2020.5638974155331
	11	2041.7857878257296
	12	1981.6171366995025
	13	1991.2197880029564
	14	2047.099925902276
	15	2003.93415039841
	16	2056.8532654558912
	17	2063.755690138678
	18	2065.6345456270355
	19	2057.5555786779114
	20	2063.9903926775596

Best k = 12

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity param

Cross validating...100%|████████████████████████████████| Time: 0:00:44




Crossvalidation Results:
	k	MSE
	1	2468.9215946156014
	2	2236.5187932997464
	3	1363.2457577614468
	4	1260.0653114790764
	5	1206.4067112507082
	6	1103.2759823205326
	7	947.1925500021966
	8	910.6574238705327
	9	876.8554365590012
	10	877.8786841941952
	11	877.5909695548985
	12	876.5205958047598
	13	880.642904753129
	14	884.0497574067717
	15	884.1371177827267
	16	883.180650805712
	17	889.6228031538425
	18	891.9389312985905
	19	899.2462818002177
	20	904.6363366260377

Best k = 12

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter (k) =

Cross validating...100%|████████████████████████████████| Time: 0:00:50


...

Cross validating...100%|████████████████████████████████| Time: 0:00:14




Crossvalidation Results:
	k	MSE
	1	1425.4262809217332
	2	1425.9019248608445
	3	527.7941970911991
	4	419.24903992749637
	5	397.0027810278634
	6	362.81349431871547
	7	361.3582692568351
	8	358.7956458530043
	9	354.932652320649
	10	348.93409939169607
	11	348.2200051389516
	12	347.55905463033
	13	346.6825601950774
	14	348.5471232436593
	15	348.8493868144945
	16	354.3649244944348
	17	356.84257552340193
	18	358.2755727597817
	19	361.2538653188203
	20	359.04829721953234

Best k = 13

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter (k) =

Cross validating...100%|████████████████████████████████| Time: 0:00:14




Crossvalidation Results:
	k	MSE
	1	880.0617780453556
	2	674.4509856546438
	3	530.7150200623353
	4	423.52859050425275
	5	399.20316823856956
	6	366.3683945434364
	7	357.37176071917213
	8	356.1899623251478
	9	356.7913274173169
	10	348.4037746063592
	11	349.90754160081246
	12	348.77631651858707
	13	350.3270670918856
	14	353.80277605645847
	15	354.56248637862535
	16	358.19045541874016
	17	360.92225692378616
	18	360.1472806883594
	19	364.53645292601607
	20	362.1857033948058

Best k = 10

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter

([8.0, 5.0, 8.0, 8.0, 7.0, 8.0, 8.0, 4.0, 5.0, 8.0, 7.0, 8.0, 7.0, 8.0, 7.0, 7.0, 8.0, 8.0, 7.0, 8.0], [200.0, 64.0, 195.0, 71.0, 48.0, 200.0, 132.0, 200.0, 68.0, 200.0, 200.0, 20.0, 21.0, 17.0, 40.0, 85.0, 23.0, 8.0, 200.0, 18.0], [12.0, 6.0, 11.0, 20.0, 20.0, 12.0, 10.0, 4.0, 6.0, 18.0, 9.0, 10.0, 9.0, 10.0, 17.0, 8.0, 11.0, 9.0, 7.0, 13.0], [7.0, 4.0, 8.0, 8.0, 7.0, 8.0, 8.0, 6.0, 6.0, 8.0, 8.0, 8.0, 7.0, 8.0, 7.0, 7.0, 8.0, 8.0, 8.0, 8.0], [101.0, 37.0, 29.0, 38.0, 14.0, 89.0, 64.0, 200.0, 48.0, 200.0, 124.0, 13.0, 18.0, 14.0, 9.0, 28.0, 12.0, 6.0, 154.0, 14.0], [12.0, 6.0, 20.0, 10.0, 8.0, 10.0, 15.0, 8.0, 10.0, 19.0, 10.0, 10.0, 9.0, 11.0, 7.0, 8.0, 10.0, 11.0, 8.0, 10.0])

In [22]:
@show mean(no_init_TP)
@show mean(yes_init_TP)
[no_init_TP yes_init_TP]

mean(no_init_TP) = 7.2
mean(yes_init_TP) = 7.35


20×2 Matrix{Float64}:
 8.0  7.0
 5.0  4.0
 8.0  8.0
 8.0  8.0
 7.0  7.0
 8.0  8.0
 8.0  8.0
 4.0  6.0
 5.0  6.0
 8.0  8.0
 7.0  8.0
 8.0  8.0
 7.0  7.0
 8.0  8.0
 7.0  7.0
 7.0  7.0
 8.0  8.0
 8.0  8.0
 7.0  8.0
 8.0  8.0

In [23]:
@show mean(no_init_iter)
@show mean(yes_init_iter)
[no_init_iter yes_init_iter]

mean(no_init_iter) = 100.5
mean(yes_init_iter) = 60.6


20×2 Matrix{Float64}:
 200.0  101.0
  64.0   37.0
 195.0   29.0
  71.0   38.0
  48.0   14.0
 200.0   89.0
 132.0   64.0
 200.0  200.0
  68.0   48.0
 200.0  200.0
 200.0  124.0
  20.0   13.0
  21.0   18.0
  17.0   14.0
  40.0    9.0
  85.0   28.0
  23.0   12.0
   8.0    6.0
 200.0  154.0
  18.0   14.0

In [24]:
@show mean(no_init_best_k)
@show mean(yes_init_best_k)
[no_init_best_k yes_init_best_k]

mean(no_init_best_k) = 11.1
mean(yes_init_best_k) = 10.6


20×2 Matrix{Float64}:
 12.0  12.0
  6.0   6.0
 11.0  20.0
 20.0  10.0
 20.0   8.0
 12.0  10.0
 10.0  15.0
  4.0   8.0
  6.0  10.0
 18.0  19.0
  9.0  10.0
 10.0  10.0
  9.0   9.0
 10.0  11.0
 17.0   7.0
  8.0   8.0
 11.0  10.0
  9.0  11.0
  7.0   8.0
 13.0  10.0

## Can we increase IHT's true positive by finding nearby SNPs?

Short answer: not really (increase is 1-2%)

In [29]:
TP, TP_LD = process_iht(0.9, verbose=false)
@show mean(TP), mean(TP_LD);

k = 10, r = 2, polygenic model

(mean(TP), mean(TP_LD)) = (0.42875, 0.43875)


In [30]:
TP, TP_LD = process_iht(0.5)
@show mean(TP), mean(TP_LD);

k = 10, r = 2, polygenic model

(mean(TP), mean(TP_LD)) = (0.42875, 0.44125)


## Conclusion:
+ mvPLINK and GEMMA have essentially identical power, about 20% higher than IHT
+ High power of mvPLINK and GEMMA also gives **extremely high** false positives (type I error)
+ QQ and Manhattan plot for GEMMA and PLINK often look insanely bad
+ **Simulated genotypes *does not* produce insane p-values for GEMMA and mvPLINK.**
+ Searching SNPs in LD with the causal SNPs does not increase true positive rate for IHT